In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.p4p import P4P
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import redis
import pendulum
import arrow
import imp
import time
import re

import types
import traceback 
import threading

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-logging')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--ignore-certificate-errors')
browser = webdriver.Chrome(chrome_options=chrome_options)

In [3]:
broker_url = 'redis://localhost:6379/0'
market = JSON.deserialize('.', 'storage', 'markets.json')['Tools']
p4p = P4P(market, market['lid'], market['lpwd'],broker_url=broker_url, headless=False, browser=browser)

. storage markets.json
.\storage\markets.json was deserialized!
E:\glitter\Tools_config [] p4p_keywords_list_recording.json
E:\glitter\Tools_config\p4p_keywords_list_recording.json was deserialized!
E:\glitter\Tools_config [] p4p_keywords_list_monitor.json
E:\glitter\Tools_config\p4p_keywords_list_monitor.json was deserialized!


In [ ]:
def crawl(self, group="all"):

    try:
        keywords = []
        # self.load_keywords('recording')
        with self.lock:
            self.load_url()
            # all_kws_count = int(self.browser.find_element_by_css_selector('a.all-kwcount span').text)
            all_kws_count = self.switch_to_group(group)

            kws_count = 0
            while True:
                css_selector = "div.keyword-manage .bp-table-main-wraper>table"
                table = self.browser.find_element_by_css_selector(css_selector)

                trs = table.find_elements_by_css_selector('tbody tr')

                has_checked = False
                for idx, tr in enumerate(trs):
                    kws_count += 1

                    print('index:', idx, len(trs), end=' > ')
                    ActionChains(self.browser).move_to_element(tr).perform()
                    id = tr.find_element_by_css_selector('td:first-child input').get_attribute('value').strip()
                    # if id not in self.keywords_list['recording']:
                    #     print('skipped_not_in_recording')
                    #     continue

                    grp = tr.find_element_by_css_selector('td:nth-child(4)').text.strip()
                    if group != 'all':
                        if group not in grp:
                            print('skipped_not_in_group', group, grp)
                            continue

                    dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()

                    print(kws, end=' > ')

                    if not self.open_price_dialog(tr):
                        print('skipped_2')
                        continue

                    [prices, sponsors] = self.find_prices_and_sponsors()

                    if prices:
                        keywords.append([dt, id, kws, grp, prices, sponsors])

                        if (id not in self.recent_prices):
                            self.recent_prices[id] = []
                        self.recent_prices[id].append(prices)
                        if len(self.recent_prices[id]) > 5:
                            self.recent_prices[id].pop(0)

                    print(' >>>>>> successful end ')

                if not self.next_page():
                    break
        if keywords:
            print('==================================')
            self.save_crawling_result(keywords)

    except Exception as e:
        print('Error: ', e)
        traceback.print_exc()

p4p.crawl = types.MethodType(crawl, p4p)

In [ ]:
def save_crawling_result(self, keywords):
    root = self.market['directory'] + '_config'
    date_str = keywords[0][0].split(' ')[0]
    fn = 'p4p_keywords_crawl_result_'+date_str+'.json.gz'
    print(fn, root)
    JSON.serialize(keywords, root, [], fn, append=True)
    
p4p.save_crawling_result = types.MethodType(save_crawling_result, p4p)

In [ ]:
def find_prices_and_sponsors(self, close=True):
    prices = []
    sponsors = None

    kws = self.browser.find_element_by_css_selector(
        '.sc-manage-edit-price-dialog span[data-role="span-keyword"]').text.strip()
    sponsors = self.find_sponsors(kws)

    while True:
        try:
            price_table_tbody_selector = ".sc-manage-edit-price-dialog table tbody,.sc-manage-edit-price-dialog p.util-clearfix"
            price_table_tbody = WebDriverWait(self.browser, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, price_table_tbody_selector)))

            if price_table_tbody.tag_name == 'p':
                prices = None
                break

            for btn in price_table_tbody.find_elements_by_css_selector('a'):
                price = btn.text.strip()
                float(price)
                prices.append(price)

            self.check_balance()
            break
        except StaleElementReferenceException:
            #             self.browser.implicitly_wait(0.5)
            time.sleep(0.5)
            continue
        except ValueError:
            prices = []
            break
    if close:
        webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
    return [prices, sponsors]
p4p.find_prices_and_sponsors = types.MethodType(find_prices_and_sponsors, p4p)

In [24]:
def find_sponsors(self, kws):
    url = 'https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&viewtype=L&CatId=&SearchText=' + re.sub(
        ' +', '+', kws)
    self.headers['Referer'] = url
    
    print('---------------')
    print(url)
    
    response = None
    while response is None:
        try:
            response = requests.get(url, headers=self.headers)
            break
        except Exception as e:
            traceback.print_exc()
            time.sleep(3)
            print('====================== retry in 3 seconds =================================')
            response = None
            continue

    top_sponsor = None
    sponsor_list = []

    result = re.search(r'_search_result_data =(.*)page.setPageData\(_search_result_data\)', response.text,
                       re.M | re.DOTALL)
    result = re.sub(',[\n\t ]*}', '}',result.group(1))
    obj = json.loads(result)
    items = obj['normalList']
    for idx, item in enumerate(items):
        company = {}
        if (item['isBrandAd']):
            top_sponsor = company
        elif (item['isP4p']):
            sponsor_list.append(company)
        else:
            break

        company['years'] = item['supplierYear']
        company['name'] = item['supplierName']
        company['url'] = item['supplierHref']
        if 'record' in item:
            company['record'] = []
            if 'transaction' in item['record']:
                company['record'].append(item['record']['transaction']['num'])
                company['record'].append('Transactions(6 months)')
                company['record'].append(item['record']['transaction']['conducted'])
            if 'responseRate' in item['record']:
                company['record'].append('Response Rate')
                company['record'].append(item['record']['responseRate'])

    print({'top_sponsor': top_sponsor, 'sponsor_list': sponsor_list})
    return {'top_sponsor': top_sponsor, 'sponsor_list': sponsor_list}
p4p.find_sponsors = types.MethodType(find_sponsors, p4p)

In [ ]:
p4p.browser.set_window_size(1920, 1200)

In [ ]:
p4p.browser.save_screenshot('screenie.png')

In [10]:
group = '0直通车'

In [ ]:
p4p.load_url()
p4p.switch_to_group(group)

In [ ]:
p4p.crawl(group=group)

In [25]:
p4p.monitor(group=group)

Task Monitor start to run with group="0直通车"
switch to group: 0直通车
index: 0 12 > axe > ---------------
https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&viewtype=L&CatId=&SearchText=axe
{'top_sponsor': None, 'sponsor_list': [{'years': '3', 'name': 'Ronix(Zhangjiagang Bonded Area) Trading Co., Ltd.', 'url': '//ronixtools.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['28', 'Transactions(6 months)', '770,000+', 'Response Rate', '94.3%']}, {'years': '6', 'name': 'Ningbo Worldsourcing I/E Co., Ltd.', 'url': '//worldsourcingcn.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['Response Rate', '45.5%']}, {'years': '1', 'name': 'KE Trading Co., Ltd.', 'url': '//yjketrading.en.alibaba.com/company_profile.html#top-nav-bar', 'record': []}, {'years': '2', 'name': 'Qingdao Glitter Tools Co., Ltd.', 'url': '//glittertools.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['14', 'Transactions(6 months)', '50,000+', 'Response Rate', '94.4%']}, {'years':

index: 7 12 > f clamp > ---------------
https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&viewtype=L&CatId=&SearchText=f+clamp
{'top_sponsor': {'years': '2', 'name': 'Suzhou Kota Hardware&Tools Co., Ltd.', 'url': '//kotatool.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['6', 'Transactions(6 months)', '100,000+', 'Response Rate', '97.7%']}, 'sponsor_list': [{'years': '5', 'name': 'Shandong Haihong Electric Power Equipment Co., Ltd.', 'url': '//sdhhdlqc.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['1', 'Transactions(6 months)', '500+', 'Response Rate', '83.3%']}, {'years': '1', 'name': 'Handan Yongnian Yifeng Industrial And Mining Power Fittings Co., Ltd.', 'url': '//yfgkdl.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['Response Rate', '100.0%']}, {'years': '11', 'name': 'Cixi Zonghan Xinyi Hardware Factory', 'url': '//china-nbxinyi.en.alibaba.com/company_profile.html#top-nav-bar', 'record': ['Response Rate', '75.0%']}, {'years'

In [ ]:
p4p.turn_all_off(group=group)

In [ ]:
float('300.00') == 300

In [ ]:
p4p.alibaba = None

In [12]:
text = '''
		"_custid": "243557396,232961259,214156002,237512884,240697261"
			},
							"sideP4p": {
						"testVersion": "",
			"showIntelMatch": true ,
			"urlPrefix": "\u002f\u002fp4p\u002denmatch.alibaba.com\u002fasync\u002fb2bad.do",
			"dcatid": "",
			"styleVersion": "",
			"offertype": "magellan011131124087-2018-09-01",
			"export_market": "",
			"imgSizeRule": "140x140",
			"clickServerDomain": "http\u003a\u002f\u002fhz.clickproxy.p4p.alibaba.com\u002fclickRedirect.do",
			"pageIndex": 1 ,
			"mt": "ec",
			"dl": "r",
			"ip": "123.168.123.1",
			"pid": "601_0000_0103",
			"server": "",
			"count": 10 ,
			"keyword": "axe",
			"domain": "\u002f\u002fwww.alibaba.com",
			"offset":  0 ,
			"country": "",
            "currentLang": ""
			},
							"bottomP4p": {
						"testVersion": "",
			"showIntelMatch": true ,
			"urlPrefix": "\u002f\u002fp4p\u002denmatch.alibaba.com\u002fasync\u002fb2bad.do",
			"dcatid": "",
			"styleVersion": "",
			"offertype": "magellan011131124087-2018-09-01",
			"export_market": "",
			"imgSizeRule": "200x200",
			"clickServerDomain": "http\u003a\u002f\u002fhz.clickproxy.p4p.alibaba.com\u002fclickRedirect.do",
			"pageIndex": 1 ,
			"mt": "ec",
			"dl": "r",
			"ip": "123.168.123.1",
			"pid": "601_0000_0102",
			"server": "",
			"count": 4 ,
			"keyword": "axe",
			"domain": "\u002f\u002fwww.alibaba.com",
			"offset": 10 ,
			"country": "",
            "currentLang": ""
			},
				"end":0,
	}

    '''
text

'\n\t\t"_custid": "243557396,232961259,214156002,237512884,240697261"\n\t\t\t},\n\t\t\t\t\t\t\t"sideP4p": {\n\t\t\t\t\t\t"testVersion": "",\n\t\t\t"showIntelMatch": true ,\n\t\t\t"urlPrefix": "//p4p-enmatch.alibaba.com/async/b2bad.do",\n\t\t\t"dcatid": "",\n\t\t\t"styleVersion": "",\n\t\t\t"offertype": "magellan011131124087-2018-09-01",\n\t\t\t"export_market": "",\n\t\t\t"imgSizeRule": "140x140",\n\t\t\t"clickServerDomain": "http://hz.clickproxy.p4p.alibaba.com/clickRedirect.do",\n\t\t\t"pageIndex": 1 ,\n\t\t\t"mt": "ec",\n\t\t\t"dl": "r",\n\t\t\t"ip": "123.168.123.1",\n\t\t\t"pid": "601_0000_0103",\n\t\t\t"server": "",\n\t\t\t"count": 10 ,\n\t\t\t"keyword": "axe",\n\t\t\t"domain": "//www.alibaba.com",\n\t\t\t"offset":  0 ,\n\t\t\t"country": "",\n            "currentLang": ""\n\t\t\t},\n\t\t\t\t\t\t\t"bottomP4p": {\n\t\t\t\t\t\t"testVersion": "",\n\t\t\t"showIntelMatch": true ,\n\t\t\t"urlPrefix": "//p4p-enmatch.alibaba.com/async/b2bad.do",\n\t\t\t"dcatid": "",\n\t\t\t"styleVersion": "

In [13]:
print(text)


		"_custid": "243557396,232961259,214156002,237512884,240697261"
			},
							"sideP4p": {
						"testVersion": "",
			"showIntelMatch": true ,
			"urlPrefix": "//p4p-enmatch.alibaba.com/async/b2bad.do",
			"dcatid": "",
			"styleVersion": "",
			"offertype": "magellan011131124087-2018-09-01",
			"export_market": "",
			"imgSizeRule": "140x140",
			"clickServerDomain": "http://hz.clickproxy.p4p.alibaba.com/clickRedirect.do",
			"pageIndex": 1 ,
			"mt": "ec",
			"dl": "r",
			"ip": "123.168.123.1",
			"pid": "601_0000_0103",
			"server": "",
			"count": 10 ,
			"keyword": "axe",
			"domain": "//www.alibaba.com",
			"offset":  0 ,
			"country": "",
            "currentLang": ""
			},
							"bottomP4p": {
						"testVersion": "",
			"showIntelMatch": true ,
			"urlPrefix": "//p4p-enmatch.alibaba.com/async/b2bad.do",
			"dcatid": "",
			"styleVersion": "",
			"offertype": "magellan011131124087-2018-09-01",
			"export_market": "",
			"imgSizeRule": "200x200",
			"clickServerDomain": "htt

In [21]:
result = re.sub(',[\n\t ]*}', '\n}',text)
print(result)


		"_custid": "243557396,232961259,214156002,237512884,240697261"
			},
							"sideP4p": {
						"testVersion": "",
			"showIntelMatch": true ,
			"urlPrefix": "//p4p-enmatch.alibaba.com/async/b2bad.do",
			"dcatid": "",
			"styleVersion": "",
			"offertype": "magellan011131124087-2018-09-01",
			"export_market": "",
			"imgSizeRule": "140x140",
			"clickServerDomain": "http://hz.clickproxy.p4p.alibaba.com/clickRedirect.do",
			"pageIndex": 1 ,
			"mt": "ec",
			"dl": "r",
			"ip": "123.168.123.1",
			"pid": "601_0000_0103",
			"server": "",
			"count": 10 ,
			"keyword": "axe",
			"domain": "//www.alibaba.com",
			"offset":  0 ,
			"country": "",
            "currentLang": ""
			},
							"bottomP4p": {
						"testVersion": "",
			"showIntelMatch": true ,
			"urlPrefix": "//p4p-enmatch.alibaba.com/async/b2bad.do",
			"dcatid": "",
			"styleVersion": "",
			"offertype": "magellan011131124087-2018-09-01",
			"export_market": "",
			"imgSizeRule": "200x200",
			"clickServerDomain": "htt

In [ ]:
# p4p.load_url()

# css_selector = "div.keyword-manage .bp-table-main-wraper>table"
# table = p4p.browser.find_element_by_css_selector(css_selector)
# trs = table.find_elements_by_css_selector('tbody tr')
# idx = 1
# for tr in trs:
#     print()
#     print(idx, end=': ')
#     idx += 1
    
#     kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
#     print(kws, end=": ")
        
#     if not p4p.open_price_dialog(tr):
#         print('skipped_2')
#         idx += 1
#         continue
        
#     [prices, sponsors] = p4p.find_prices_and_sponsors()
#     print(prices, sponsors)
#     if idx > 10:
#         break

In [ ]:
tr = trs[8]
kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
print(kws, end=": ")

if not p4p.open_price_dialog(tr):
    print('skipped_2')
else:
#     time.sleep(1)
    prices = p4p.find_prices()

In [ ]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Tools']

In [ ]:
def set_sub_budget(self, sub_budget):
    self.redis.set(self.market['name']+'_p4p_sub_budget', sub_budget)

def unset_sub_budget(self):
    self.redis.delete(self.market['name']+'_p4p_sub_budget')

In [ ]:
diff = '19.63'
diff = 0-float(diff)
sub_budget = self.redis.get(self.market['name']+'_p4p_sub_budget')
if sub_budget is not None and float(sub_budget) > 0:
    if self.redis.incrbyfloat(self.market['name']+'_p4p_sub_budget', diff) < 0:
        self.redis.set(self.market['name']+'_p4p_sub_budget_overflow', True)

In [ ]:
r = redis.StrictRedis.from_url(p4p.broker_url)

In [ ]:
sub_budget = '90.00'
r.set(p4p.market['name']+'_p4p_sub_budget', sub_budget)

In [ ]:
float(r.get(p4p.market['name']+'_p4p_sub_budget'))

In [ ]:
r.get(p4p.market['name']+'_p4p_sub_budget')

In [ ]:
r.incrbyfloat(p4p.market['name']+'_p4p_sub_budget', -19.63)

In [ ]:
r.delete(p4p.market['name']+'_p4p_sub_budget')

In [ ]:
diff = '19.63'
diff = 0-float(diff)
diff

In [ ]:
r.delete(p4p.market['name']+'_p4p_sub_budget_overflow')